In [ ]:
#We'll use matrix decomposition from linear algebra to do some image processing


In [34]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import misc

In [53]:
# img = misc.face()
img = cv2.imread('face.png')


In [ ]:
# show the image
plt.imshow(img)

In [ ]:
# get the shape, axis and array properties
print(img.shape)
print(type(img))
print(img.ndim)

In [ ]:
# the three dimensions represent the red green blue color channels of the image
img[:,:,0] # red channel


In [ ]:
plt.imshow(img[:,:,0])

In [ ]:
print(img.max(), img.min()) # max and min values in the image

In [58]:
# let's convert the values into floats for better linear algebra operations
# Normalizing...
img_array = img/255

In [ ]:
print(img_array.max(), img_array.min(), img_array.dtype)

In [60]:
# assign each color to a seperate matrix using array slicing

red_channel = img_array[:,:,0]
green_channel = img_array[:,:,1]
blue_channel = img_array[:,:,2]

In [48]:
# from linear algebra, it's possible to approximate existing sets of data
# we'll use the SVF (Singular Value decomposition)
#SVD returns 3 matrices which can be multiplied to a given matrix A
# U.S.V_transpose = A
#U and V are square, and S is same size as A
# S is a diagonal matrix and contains the singular values of A
# The values on the diagonal are always non-negative 
# and is an indicator of the importance of some features in A


In [61]:
# According to colorimetry you can obtain a grayscale image
# of a color image using the following formula
# Y = 0.2126R + 0.7152G + 0.0722B

# img_gray = img_array @ [0.2126, 0.7152, 0.0722]
img_gray=np.matmul(img_array,[0.2126, 0.7152, 0.0722])

In [62]:
img_gray.shape

(768, 1024)

In [ ]:
plt.imshow(img_gray, cmap='gray')

In [64]:
# applying the SVD to obtain the decomposition
U,s,Vt = np.linalg.svd(img_gray)

In [ ]:
# let's confirm the shapes of the matrices
print(U.shape, s.shape, Vt.shape)

In [70]:
# peculiar shape of s is for space saving. Let's reconstruct the original matrix
Sigma = np.zeros((768,1024))
for i in range(768):
    Sigma[i,i] = s[i] # values are only in the diagonal


In [ ]:
# how close is the reconstructed U . Sigm . Vt to original grayscale image?
# we can do check this using the norm of the difference

# print(np.linalg.norm(img_gray - U @ Sigma @ Vt))
print(np.linalg.norm(img_gray - np.matmul(np.matmul( U , Sigma) , Vt)))

In [ ]:
# Let's see which singular values are important in building an approximation for our image
plt.plot(s)
plt.xlabel('s')
plt.ylabel('i')
plt.title('SVD')

In [ ]:
# the idea is to consider all but the first k singuular values in Sigma as 0 and keep U and Vt intact
# e.g k = 10

k =30
approx = U @ Sigma[:,:k] @ Vt[:k,:]
plt.imshow(approx, cmap = 'gray')

In [81]:
Sigma[:,:k].shape

(768, 10)

In [90]:
from datetime import datetime, date, time, timedelta
import dateutil

In [ ]:
# let's create our own database

# read first names from text file
# read last names from text file


In [88]:
# create a random date range
start_date = date(1950,1,1)
end_date = date(2023,4,6)
date_array = np.arange(start_date,end_date)

In [89]:
len(date_array)

26758

In [91]:
def get_user_details():
    name = input("Name: ")
    dob = input("DOB: mm/dd/yyyy")
    return name, dateutil.parser.parse(dob)

a,b = get_user_details()

In [97]:
# convert from datetime to a timestamp
# find the difference between times
datetime.timestamp(b)
b - datetime.now()

datetime.timedelta(days=-12569, seconds=54039, microseconds=322660)

In [107]:
#convert numpy datetime64 to datetime
date_array[0].astype(datetime).year

1950